<img src="../images/AzPTravel_PPM.png">

## Finance Comparison Report

#### This script reads the output of the Upload Assistant and summarizes all finance views with comments along with each report.

#### This can be run on its own or as part of the pipeline wherever called (ie from a papermill script or other user inteface).

### User Variables
- These are overwritten if inherited from run_control.ipynb.
- Feel Free to reset them for a manual run if you like
- Do not save without percode = "-f"

In [1]:
commit_message = "Development and testing."
# Give a brief reason for the run.

run_control = 1
#run_type = 0 - Lite run with no reporting, not recommended.
#run_type = 1 - Lite run with normal reporting, default setting.
#run_type = 2 - Heavy run with full reporting, available for audits and troubleshooting.
#run_type = 5 - A default setting. Indicates the script is being run by an outside process without an inherited value

percode = "2021.Q1"
# Data Collection Code, this controls file paths and output names
# "-f" is the value indicating a bad inheritance from run with arg

s_format = "p"
# denotes the source data format x == Excel; j == json, p == parquet

#----------
# do not edit - this either inherits the full instance timestamp from the papermill book or captures the run time of this script.
from datetime import datetime
inst_datetime = datetime.now().strftime("%m%d%Y%H%M%S")

In [2]:
# Parameters
run_control = 1
percode = "2021.Q1"
commit_message = "Live run, added AT, bad file name, rerun."
inst_datetime = "06012021173656"


#### Notebook display options

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
#### import packages

In [5]:
#### Packages used

import os
import sys

from datetime import datetime
import warnings

import glob
import getpass
import json

import matplotlib.pyplot as plt
import openpyxl
import numpy as np
import pandas as pd

import re
import nicexcel as nl
import xlsxwriter

global df

In [6]:
#### Default Variables, these govern logic, do not edit.

In [7]:
default_dc = "20XX.QX"
default_rc = 0 #extra lite mode
dummy_perc = "33Q3" # bad inheritance

In [8]:
#### Script determining run context ie, manual, run_control.ipynb, or other.

In [9]:
if run_control == 5:
    run_control = default_rc 
else:
    run_control = run_control

try:
    if sys.argv[1] == "-f":
        percode = percode
    else:
        percode = sys.argv[1]

except IndexError:
    percode = default_dc
except NameError:
    percode = default_dc


#### style settings

In [10]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Begin Input Read


#### Make paths used in script
- all paths should be modular and easily replaced ahead of operations

In [11]:
#root directory
rt_path = f'//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production/{str(percode)}'
#live sources directory
ls_path = os.path.join( rt_path, 'live_sources')
#financial report input dir
fs_path = os.path.abspath(os.path.join(rt_path, '..','..',f"{percode}", 'Finance Report'))
#report ouput dir
rep_path = os.path.abspath(os.path.join(rt_path, '..','..',f"{percode}", 'Standard Reports'))
#report ouput archive dir
rep_arch_path = os.path.abspath(os.path.join(rep_path, '..','..',f"{percode}",  'Archive'))
#r project path
rscript_path = os.path.abspath(os.path.join(rt_path, '..','..', 'Pipeline Reporting Scripts','Finance Comparison PnL'))



#### Make sources used in script
- all sources should be modular and easily replaced ahead of operations

In [12]:
fin_xls = os.path.join(fs_path ,"Finance_Report.xlsx"   )

rscript = os.path.join( rscript_path, 'Finance_Comparison_PnL.Rmd')

#### Get a list of only source files in the path that start with "us_dat".
#### Logic determines the source file types.

#### User instructions:
- Make sure that you have 1 file per source in this folder.
    -  For instance, do not have two files for Portugal. If there is an update, archive the old one.
- Do not overwrite files in the archive.
    - Rename newly archived files, no strict convention, we keep track of these by the modified date.
- It is ok to have multiple sources in one file.

In [13]:
files = os.listdir(ls_path)
files = [files.lower() for files in files]

files_sour = [f for f in files if f[-5:]  == '.json' or  f[-8:] == '.parquet' and f[:7] != 'us_orig']

files_sour

['us_dat_at_+te_q1_2021_at.parquet',
 'us_dat_au_05142021143612.parquet',
 'us_dat_ca_05202021134242.parquet',
 'us_dat_ch_05182021192455.parquet',
 'us_dat_cz_05262021090243.parquet',
 'us_dat_es_05202021105141.parquet',
 'us_dat_gr_05252021115404.parquet',
 'us_dat_it_05242021111520.parquet',
 'us_dat_nl_05272021163120.parquet',
 'us_dat_pl_05262021084450.parquet',
 'us_dat_pt_05202021121056.parquet',
 'us_orig_au_05142021143611.json',
 'us_survey_au_02172021174808.parquet',
 'us_survey_au_05142021143750.parquet',
 'us_survey_ca_05202021135610.parquet',
 'us_survey_ch_05182021192905.parquet',
 'us_survey_cz_05262021090440.parquet',
 'us_survey_es_05202021105431.parquet',
 'us_survey_gr_05252021115827.parquet',
 'us_survey_it_05242021111759.parquet',
 'us_survey_nl_05272021163630.parquet',
 'us_survey_pl_05262021084708.parquet',
 'us_survey_pt_05202021121304.parquet',
 'us_vcomments_au_02172021174434.parquet',
 'us_vcomments_au_02172021174440.parquet',
 'us_vcomments_au_05142021143612

In [14]:
list_jfiles = glob.glob(os.path.join(ls_path, 'us_vcomments_*.json'))
list_pfiles = glob.glob(os.path.join(ls_path, 'us_vcomments_*.parquet'))

for idx, i in enumerate(list_jfiles):
    list_jfiles[idx] = list_jfiles[idx][-23:-19]
    list_jfiles[idx] = list_jfiles[idx].replace('_','')

for idx, i in enumerate(list_pfiles):
    list_pfiles[idx] = list_pfiles[idx][-26:-22]
    list_pfiles[idx] = list_pfiles[idx].replace('_','')

list_files = []
    
for i in list_jfiles : list_files.append(i)    
for i in list_pfiles : list_files.append(i)


BU_set = set(list_files)
BU_list = list(BU_set)

BU_list

['CH', 'IT', 'NL', 'CA', 'PT', 'PL', 'CZ', 'ES', 'GR', 'AU']

In [15]:
files_cmnts = [i for i in files_sour if "vcomments" in i  and "parq" in i ]


In [16]:
cols = ['Business Unit', 'Validation Rule', 'Comments', 'Row Counts', 'Check Type']


latest_valcomments = pd.DataFrame(columns = cols)


for idx, i in enumerate(BU_list):
    list_files_a = glob.glob(os.path.join(ls_path, f'us_vcomments_{i}*.parquet'))
    recent_vers_a = max(list_files_a, key=os.path.getctime)
    idf  =  pd.read_parquet(recent_vers_a, engine = "pyarrow")
    idf.insert(0, "Business Unit",BU_list[idx])
    latest_valcomments = latest_valcomments.append(idf)

latest_valcomments = latest_valcomments.reset_index(drop=True)

latest_valcomments 


,Business Unit,Validation Rule,Comments,Row Counts,Check Type,Message
0,CH,Finance Reporting.,Where do those financial figures come from?,1,Written Revenue,NaN
1,CH,Finance Reporting.,Where do those financial figures come from?,1,Earned Revenue,NaN
2,CH,Finance Reporting.,Where do those financial figures come from?,1,Loss Ratio,NaN
3,CH,Finance Reporting.,Where do those financial figures come from?,1,Commission Ratio,NaN
4,CH,Finance Reporting.,Where do those financial figures come from?,1,Expense Ratio,NaN
...,...,...,...,...,...,...
100,AU,Number of Policies (Written),Group Policies under Financial institutions do...,38,Row Check,The following 38 rows are missing entries for ...
101,AU,Commission Ratio,Result of refunds due to COVID 19,1,Row Check,"The following row has a ""Commission Ratio"" not..."
102,AU,Expense Ratio,Expense Ratio does not need to be between 0% a...,37,Row Check,"The following 37 rows have a ""Expense Ratio"" n..."
103,AU,Reporting Date From,"No updated Upload Assistant has been provided,...",53,Row Check,"""Reporting Date From"" is incorrect on 53 rows,..."


In [17]:
latest_vdfdata_sets = []

for idx, i in enumerate(BU_list):
    try:
        list_files_b = glob.glob(os.path.join(ls_path, f'us_vdf_{i}*.json'))
        recent_vers_b = max(list_files_b, key=os.path.getctime)
        with open(recent_vers_b, 'r') as fp:
            data_dict = json.load(fp)
            dfslist =   { key: pd.DataFrame(data_dict[key]) for key in data_dict  }
            for i in dfslist: latest_vdfdata_sets.append(dfslist[i])
    except ValueError:
        pass

In [18]:
fin_df = pd.read_excel(fin_xls, sheet_name= 'Finance_Report' ,  nrows=60)

fin_df = fin_df[~fin_df["DC_BU_Code"].isna()]

# still sllooooowwwwww


## End Input Read
## Begin Transformations

In [19]:
latest_valcomments["Applies to"] = latest_valcomments["Business Unit"]


In [20]:
for idx, i in enumerate(latest_valcomments["Business Unit"]):
    if "+" in  latest_valcomments["Business Unit"][idx]:
        latest_valcomments.iat[idx , 5]  = "Multiple Business Units"
    else:
        latest_valcomments.iat[idx , 5]   = "Business Unit Only"


In [21]:
for idx, i  in  enumerate(latest_vdfdata_sets):
            c = len(i.columns)
            i.insert(c , "Submission Comment", latest_valcomments["Comments"][idx])
            i.insert(c +1, "Applies to", latest_valcomments["Applies to"][idx])


## End Transformations
## Begin Report Making



In [22]:
wrcols = ['Country', '(LC) Submission Written Revenue net of Taxes',
       '(EUR) Submission Written Revenue net of Taxes',
       '(EUR) Finance Sales Gross Written', '(EUR) Difference', "Submission Comment","Applies to" ]

ercols = ['Country', '(LC): Submission Earned Revenues net of Taxes',
       '(EUR): Submission Earned Revenues net of Taxes',
       '(EUR) Finance Net Earnings', '(EUR) Difference', "Submission Comment","Applies to" ]

lrcols = ['Country', 'Submission Loss Ratio', 'Finance Loss Ratio',
       'Difference (%)', "Submission Comment","Applies to" ]

crcols = ['Country', 'Submission Commission Ratio', 'Finance Commission Ratio',
       'Difference (%)', "Submission Comment","Applies to" ]

etcols = ['Country', 'Submission Expense Ratio', 'Finance Expense Ratio',
       'Difference (%)', "Submission Comment","Applies to" ]

Written_Revenue_Report = pd.DataFrame(columns = wrcols )
Earned_Revenue_Report = pd.DataFrame(columns = ercols )
Loss_Ratio_Report = pd.DataFrame(columns = lrcols )
Commission_Ratio_Report = pd.DataFrame(columns = crcols )
Expense_Ratio_Report = pd.DataFrame(columns = etcols )


for idx, i in enumerate(latest_vdfdata_sets):
    try:
        Written_Revenue_Report = Written_Revenue_Report.append(i[wrcols])
    except:
        pass
    try:
        Earned_Revenue_Report = Earned_Revenue_Report.append(i[ercols])
    except:
        pass
    try:
        Loss_Ratio_Report = Loss_Ratio_Report.append(i[lrcols])
    except:
        pass
    try:
        Commission_Ratio_Report = Commission_Ratio_Report.append(i[crcols])
    except:
        pass
    try:
        Expense_Ratio_Report = Expense_Ratio_Report.append(i[etcols])
    except:
        pass

In [23]:
fin_rep_dict = { "Written Revenue Comparison" : Written_Revenue_Report , "Earned Revenue Comparison" : 
                Earned_Revenue_Report , "Loss Ratio Comparison" : Loss_Ratio_Report,
                "Commission Ratio Comparison" : Commission_Ratio_Report, "Expense Ratio Comparison"  :
                Expense_Ratio_Report  }

fin_rep_dict

{'Written Revenue Comparison':           Country (LC) Submission Written Revenue net of Taxes  \
 0     Switzerland                                  30,066,388    
 0           Italy                                   1,218,961    
 0     Netherlands                                   9,053,909    
 0          Canada                                  10,006,969    
 0        Portugal                                      57,474    
 0          Poland                                   6,475,634    
 0  Czech Republic                                     379,669    
 0           Spain                                     892,391    
 0          Greece                                     137,576    
 0       Australia                                     141,482    
 
   (EUR) Submission Written Revenue net of Taxes  \
 0                                   27,086,836    
 0                                    1,218,961    
 0                                    9,053,909    
 0                     

In [24]:
for k, v  in fin_rep_dict.items():
    v = pd.DataFrame(v)
    cols = v.columns
    newcols = []
    for c in cols : newcols.append( f"{k} - {c}")
    for c in cols : v.columns=(newcols)

            #: fin_rep_dict[k].update(v.rename(columns={i: f"{k} -{i}"}))
fin_rep_dict


{'Written Revenue Comparison':   Written Revenue Comparison - Country  \
 0                          Switzerland   
 0                                Italy   
 0                          Netherlands   
 0                               Canada   
 0                             Portugal   
 0                               Poland   
 0                       Czech Republic   
 0                                Spain   
 0                               Greece   
 0                            Australia   
 
   Written Revenue Comparison - (LC) Submission Written Revenue net of Taxes  \
 0                                        30,066,388                           
 0                                         1,218,961                           
 0                                         9,053,909                           
 0                                        10,006,969                           
 0                                            57,474                           
 0             

In [25]:
for k, v  in fin_rep_dict.items():
    v = pd.DataFrame(v)
    fin_df =  pd.merge(fin_df, v, left_on= "BU_Desc", right_on= f"{k} - Country", how= 'left')

fin_df

,DC_BU_Code,BU_Desc_ Finance,BU_Desc,Total SGW,Total NER,Commissions Acquis in %,Claims Costs (LAE & FA) in %,FA Acquis in %,FA Acquis in % -Second,Non manageable in %,...,Commission Ratio Comparison - Finance Commission Ratio,Commission Ratio Comparison - Difference (%),Commission Ratio Comparison - Submission Comment,Commission Ratio Comparison - Applies to,Expense Ratio Comparison - Country,Expense Ratio Comparison - Submission Expense Ratio,Expense Ratio Comparison - Finance Expense Ratio,Expense Ratio Comparison - Difference (%),Expense Ratio Comparison - Submission Comment,Expense Ratio Comparison - Applies to
0,AU,BU Australia,Australia,-3142.85,6328.70,0.239891,0.238371,0.111987,0.226514,-0.010236,...,24.0%,-21.7%,As a consequence of renegotiating some of the ...,AU,Australia,51.8%,56.7%,-4.9%,There is actually two reasons for the Expense ...,AU
1,CN,BU China,China,201.90,203.73,0.411083,3.483679,1.335346,1.023462,-0.310607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MY,BU Malaysia,Malaysia,-0.40,6.93,0.558442,8.571429,1.340548,2.900433,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NZ,BU New Zealand,New Zealand,1334.69,727.09,0.295851,0.179991,0.270242,0.353051,0.032816,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SG,BU Singapore,Singapore,789.92,830.24,0.075448,0.087673,0.165615,0.164699,0.271632,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CA,BU Canada,Canada,15157.61,15865.99,0.441831,0.385813,0.108502,0.069653,-0.015961,...,44.2%,-15.9%,Offline roster businesses are not included in ...,CA,Canada,22.4%,54.8%,-32.4%,Offline roster businesses are not included in ...,CA
6,US,BU United States,United States,145310.31,98303.27,0.476537,0.105567,0.139841,0.098594,0.030634,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,BR,BU Brazil,Brazil,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MX,BU Mexico,Mexico,1465.76,1465.76,0.045437,0.000000,0.000000,0.109384,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BE,BU Belgium,Belgium,4243.88,3703.48,0.138294,0.213737,0.075964,0.071409,0.000489,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
print(fin_df.columns)

Index(['DC_BU_Code', 'BU_Desc_ Finance', 'BU_Desc', 'Total SGW', 'Total NER',
       'Commissions Acquis in %', 'Claims Costs (LAE & FA) in %',
       'FA Acquis in %', 'FA Acquis in % -Second', 'Non manageable in %',
       'Loss a.y', 'Loss ratio in %', 'Total Combined ratio in %',
       'Technical Result', 'Operating Profit', 'Combined Ratio Calculation',
       'Check', 'Written Revenue Comparison - Country',
       'Written Revenue Comparison - (LC) Submission Written Revenue net of Taxes',
       'Written Revenue Comparison - (EUR) Submission Written Revenue net of Taxes',
       'Written Revenue Comparison - (EUR) Finance Sales Gross Written',
       'Written Revenue Comparison - (EUR) Difference',
       'Written Revenue Comparison - Submission Comment',
       'Written Revenue Comparison - Applies to',
       'Earned Revenue Comparison - Country',
       'Earned Revenue Comparison - (LC): Submission Earned Revenues net of Taxes',
       'Earned Revenue Comparison - (EUR): Sub

In [27]:

conv_cols = [
       'Written Revenue Comparison - (LC) Submission Written Revenue net of Taxes',
       'Written Revenue Comparison - (EUR) Submission Written Revenue net of Taxes',
       'Written Revenue Comparison - (EUR) Finance Sales Gross Written',
       'Written Revenue Comparison - (EUR) Difference',
       'Earned Revenue Comparison - (LC): Submission Earned Revenues net of Taxes',
       'Earned Revenue Comparison - (EUR): Submission Earned Revenues net of Taxes',
       'Earned Revenue Comparison - (EUR) Finance Net Earnings',
       'Earned Revenue Comparison - (EUR) Difference',
       'Loss Ratio Comparison - Submission Loss Ratio',
       'Loss Ratio Comparison - Finance Loss Ratio',
       'Loss Ratio Comparison - Difference (%)',
       'Commission Ratio Comparison - Submission Commission Ratio',       
        "Commission Ratio Comparison - Finance Commission Ratio","Commission Ratio Comparison - Difference (%)",  "Expense Ratio Comparison - Submission Expense Ratio", "Expense Ratio Comparison - Finance Expense Ratio", "Expense Ratio Comparison - Difference (%)"]


conv_cols_prc = [
        'Loss Ratio Comparison - Submission Loss Ratio',
       'Loss Ratio Comparison - Finance Loss Ratio',
       'Loss Ratio Comparison - Difference (%)',
       'Commission Ratio Comparison - Submission Commission Ratio',       
        "Commission Ratio Comparison - Finance Commission Ratio","Commission Ratio Comparison - Difference (%)", 
    "Expense Ratio Comparison - Submission Expense Ratio", "Expense Ratio Comparison - Finance Expense Ratio", 
    "Expense Ratio Comparison - Difference (%)"]




In [28]:

for i in conv_cols:
    try:
        fin_df[i] = fin_df[i].str.strip()
        fin_df[i] = fin_df[i].str.replace(",","")
        fin_df[i] = fin_df[i].str.replace("%","").fillna(0)
        fin_df[i] = pd.to_numeric(fin_df[i])
        fin_df[i] = fin_df[i] / 1000
    except AttributeError:
        pass

    
for i in conv_cols_prc:
    try:
        fin_df[i] = fin_df[i] *10
    except AttributeError:
        pass
fin_df
            
            

,DC_BU_Code,BU_Desc_ Finance,BU_Desc,Total SGW,Total NER,Commissions Acquis in %,Claims Costs (LAE & FA) in %,FA Acquis in %,FA Acquis in % -Second,Non manageable in %,...,Commission Ratio Comparison - Finance Commission Ratio,Commission Ratio Comparison - Difference (%),Commission Ratio Comparison - Submission Comment,Commission Ratio Comparison - Applies to,Expense Ratio Comparison - Country,Expense Ratio Comparison - Submission Expense Ratio,Expense Ratio Comparison - Finance Expense Ratio,Expense Ratio Comparison - Difference (%),Expense Ratio Comparison - Submission Comment,Expense Ratio Comparison - Applies to
0,AU,BU Australia,Australia,-3142.85,6328.70,0.239891,0.238371,0.111987,0.226514,-0.010236,...,0.240,-0.217,As a consequence of renegotiating some of the ...,AU,Australia,0.518,0.567,-0.049,There is actually two reasons for the Expense ...,AU
1,CN,BU China,China,201.90,203.73,0.411083,3.483679,1.335346,1.023462,-0.310607,...,0.000,0.000,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN
2,MY,BU Malaysia,Malaysia,-0.40,6.93,0.558442,8.571429,1.340548,2.900433,0.000000,...,0.000,0.000,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN
3,NZ,BU New Zealand,New Zealand,1334.69,727.09,0.295851,0.179991,0.270242,0.353051,0.032816,...,0.000,0.000,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN
4,SG,BU Singapore,Singapore,789.92,830.24,0.075448,0.087673,0.165615,0.164699,0.271632,...,0.000,0.000,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN
5,CA,BU Canada,Canada,15157.61,15865.99,0.441831,0.385813,0.108502,0.069653,-0.015961,...,0.442,-0.159,Offline roster businesses are not included in ...,CA,Canada,0.224,0.548,-0.324,Offline roster businesses are not included in ...,CA
6,US,BU United States,United States,145310.31,98303.27,0.476537,0.105567,0.139841,0.098594,0.030634,...,0.000,0.000,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN
7,BR,BU Brazil,Brazil,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN
8,MX,BU Mexico,Mexico,1465.76,1465.76,0.045437,0.000000,0.000000,0.109384,0.000000,...,0.000,0.000,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN
9,BE,BU Belgium,Belgium,4243.88,3703.48,0.138294,0.213737,0.075964,0.071409,0.000489,...,0.000,0.000,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN


#### Output to r project

In [29]:
outfile =  os.path.join(rscript_path, "finrep.parquet")

fin_df.to_parquet(outfile, engine = "pyarrow" )